# Comparing different DFO for Privugger-AG
*MADE BY: Mathias Oliver Valdbjørn Jørgensen mathiasoliverjorgensen@hotmail.com*


This document contains a comparison of different derivative free optimization technique for privacy risk analysis.

# Imports

In [1]:
import numpy as np
import opendp.smartnoise.core as sn
from sklearn.feature_selection import mutual_info_regression
from typing import List, Tuple
from scipy import stats as st
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns
import re
import subprocess
from entropy_estimators import continuous
from GPyOpt.methods import BayesianOptimization #Bayesian Optimization
import pybobyqa # For Bobyqa
from scipy import optimize # For Powell
import pyswarms as ps

## Random Search

In [2]:
def random_search(f, domain, iterations=10_000):
    def val():
        v = []
        for d in domain:
            s = np.random.uniform(d["domain"][0],d["domain"][1])
            v.append(s)
        return np.array(v)
    best = np.inf
    x_best = []
    for i in range(iterations):
        x = val()
        b = f(x)
        if b < best:
            best = b
            x_best = x
    return best, x_best

## Powell

In [3]:
def powell(f, domain):
    v = []
    for d in domain:
        v.append(np.random.uniform(d["domain"][0],d["domain"][1]))
    bounds = [d["domain"] for d in domain]
    
    return optimize.minimize(f, 
            v, 
            method="Powell", 
            bounds=bounds,
            options={'xtol': 0.0000001, 'ftol': 0.000001})

## Bobyqa

In [4]:
def boby(f, domain):
    x0 = []
    for d in domain:
        x0.append(np.random.uniform(d["domain"][0],d["domain"][1]))
    lower_bounds = [d["domain"][0] for d in domain]
    upper_bounds = [d["domain"][1] for d in domain]
    return pybobyqa.solve(f, x0, bounds=(lower_bounds, upper_bounds))

## Particle Swarm

In [5]:
def particle_swarm(f,domain, n_particles=30):
    lower_bounds = [d["domain"][0] for d in domain]
    upper_bounds = [d["domain"][1] for d in domain]
    bounds = (np.array(lower_bounds), np.array(upper_bounds))
    options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
    optimizer = ps.single.GlobalBestPSO(n_particles=n_particles, dimensions=len(domain), options=options, bounds=bounds)

    return optimizer.optimize(f, 180)

## Bayesian optimization

In [6]:
def b_opt(f, domain):
    INITIAL_DESIGN_NUMDATA = 10
    MAX_ITER = 50
    ds = []
    for i,d in enumerate(domain):
        ds.append({"name": f"alice_{i}", "type": "continuous", "domain": d["domain"]})
    Bopt =  BayesianOptimization(f=f, domain=ds, 
                             acquisition_type='EI',        # Expected Improvement
                             initial_design_numdata=INITIAL_DESIGN_NUMDATA,
                             exact_feval = True)
    Bopt.run_optimization(max_iter = MAX_ITER-INITIAL_DESIGN_NUMDATA, eps=1e-8) 
    return (Bopt.fx_opt, Bopt.x_opt)

# Evaluations


In [7]:
def evaluate(f, domain, best_f, best_x,eps=0.02):
    reachesMax = [False]*5
    accuracy = [0]*5
    for i in range(100):
        f1, x1 = random_search(f,domain, iterations=1_000)

        temp = powell(f,domain)
        f2 = temp.fun
        x2 = temp.x

        temp = boby(f, domain)
        f3, x3 = temp.f, temp.x
        
        f_part = lambda x: np.array([f(xi) for xi in x])
        f4, x4 = particle_swarm(f_part,domain)
        
        f_bo = lambda x: f(x[0])
        f5,x5 = b_opt(f_bo,domain)
        
        for i,res in enumerate([f1,f2,f3,f4,f5]):
            if best_f-eps <= res <= eps+best_f:
                accuracy[i] += 1
                reachesMax[i] = True
            
    return accuracy, reachesMax

## Binomial entropy

In [105]:
def bern_entropy(x):
    p = x[0]
    return -np.log2(np.exp(st.bernoulli(p).entropy()))

maximum_x = [0.5]
maximum_f = -1

parameters_domain = [
    {"domain": (0,1)}
]
evaluate(bern_entropy, parameters_domain, maximum_f, maximum_x)

2021-05-26 18:35:03,499 - root - INFO - Function eval 1 at point 1 has f = -0.147311382129051 at x = [0.97894724]
2021-05-26 18:35:03,500 - root - INFO - Initialising (coordinate directions)
2021-05-26 18:35:03,503 - root - INFO - Function eval 2 at point 2 has f = -0 at x = [1.]
2021-05-26 18:35:03,505 - root - INFO - Function eval 3 at point 3 has f = -0.532379441663222 at x = [0.87894724]
2021-05-26 18:35:03,506 - root - INFO - Beginning main loop
2021-05-26 18:35:03,508 - root - INFO - New rho = 0.01 after 3 function evaluations
2021-05-26 18:35:03,511 - root - INFO - Function eval 4 at point 4 has f = -0.597500453352088 at x = [0.85487675]
2021-05-26 18:35:03,515 - root - INFO - Function eval 5 at point 5 has f = -0.803456756055352 at x = [0.75487675]
2021-05-26 18:35:03,519 - root - INFO - Function eval 6 at point 6 has f = -0.92690258527698 at x = [0.65780618]
2021-05-26 18:35:03,523 - root - INFO - Function eval 7 at point 7 has f = -0.958607422231234 at x = [0.61919666]
2021-0

2021-05-26 18:35:44,419 - root - INFO - Function eval 3 at point 3 has f = -0.0459916345194312 at x = [0.00507566]
2021-05-26 18:35:44,420 - root - INFO - Beginning main loop
2021-05-26 18:35:44,423 - root - INFO - Function eval 4 at point 4 has f = -0.860701916231689 at x = [0.28389352]
2021-05-26 18:35:44,425 - root - INFO - New rho = 0.01 after 4 function evaluations
2021-05-26 18:35:44,428 - root - INFO - Function eval 5 at point 5 has f = -0.907171732258683 at x = [0.32258622]
2021-05-26 18:35:44,431 - root - INFO - Function eval 6 at point 6 has f = -0.992260449125828 at x = [0.4482552]
2021-05-26 18:35:44,434 - root - INFO - Function eval 7 at point 7 has f = -0.997674839455562 at x = [0.47162032]
2021-05-26 18:35:44,438 - root - INFO - Function eval 8 at point 8 has f = -0.999802422289007 at x = [0.50827479]
2021-05-26 18:35:44,441 - root - INFO - Function eval 9 at point 9 has f = -0.999999634126961 at x = [0.49964391]
2021-05-26 18:35:44,444 - root - INFO - Function eval 10 a

2021-05-26 18:36:22,884 - root - INFO - New rho = 0.01 after 4 function evaluations
2021-05-26 18:36:22,887 - root - INFO - Function eval 5 at point 5 has f = -0.999714585688594 at x = [0.50994538]
2021-05-26 18:36:22,891 - root - INFO - Function eval 6 at point 6 has f = -0.99999966333419 at x = [0.50034158]
2021-05-26 18:36:22,894 - root - INFO - Function eval 7 at point 7 has f = -0.999074349086107 at x = [0.48209085]
2021-05-26 18:36:22,895 - root - INFO - New rho = 0.001 after 7 function evaluations
2021-05-26 18:36:22,898 - root - INFO - Function eval 8 at point 8 has f = -0.999996128010304 at x = [0.49884158]
2021-05-26 18:36:22,901 - root - INFO - Function eval 9 at point 9 has f = -0.99999021437717 at x = [0.50184158]
2021-05-26 18:36:22,903 - root - INFO - New rho = 0.0001 after 9 function evaluations
2021-05-26 18:36:22,906 - root - INFO - Function eval 10 at point 10 has f = -1 at x = [0.5]
2021-05-26 18:36:22,910 - root - INFO - Function eval 11 at point 11 has f = -0.9999

2021-05-26 18:37:05,013 - root - INFO - Function eval 9 at point 9 has f = -0.999999943858832 at x = [0.49986051]
2021-05-26 18:37:05,015 - root - INFO - New rho = 1e-05 after 9 function evaluations
2021-05-26 18:37:05,018 - root - INFO - Function eval 10 at point 10 has f = -1 at x = [0.5]
2021-05-26 18:37:05,021 - root - INFO - Function eval 11 at point 11 has f = -0.999999999350787 at x = [0.500015]
2021-05-26 18:37:05,023 - root - INFO - New rho = 1e-06 after 11 function evaluations
2021-05-26 18:37:05,026 - root - INFO - Function eval 12 at point 12 has f = -0.999999999993508 at x = [0.4999985]
2021-05-26 18:37:05,028 - root - INFO - Function eval 13 at point 13 has f = -0.999999999993508 at x = [0.5000015]
2021-05-26 18:37:05,030 - root - INFO - New rho = 1e-07 after 13 function evaluations
2021-05-26 18:37:05,033 - root - INFO - Function eval 14 at point 14 has f = -0.999999999999935 at x = [0.50000015]
2021-05-26 18:37:05,036 - root - INFO - Function eval 15 at point 15 has f =

2021-05-26 18:37:45,205 - root - INFO - Function eval 14 at point 14 has f = -0.999999999995511 at x = [0.50000125]
2021-05-26 18:37:45,207 - root - INFO - New rho = 1e-07 after 14 function evaluations
2021-05-26 18:37:45,211 - root - INFO - Function eval 15 at point 15 has f = -0.999999999999935 at x = [0.49999985]
2021-05-26 18:37:45,214 - root - INFO - Function eval 16 at point 16 has f = -0.999999999999955 at x = [0.50000013]
2021-05-26 18:37:45,215 - root - INFO - New rho = 1e-08 after 16 function evaluations
2021-05-26 18:37:45,218 - root - INFO - Function eval 17 at point 17 has f = -1 at x = [0.49999999]
2021-05-26 18:37:45,221 - root - INFO - Function eval 18 at point 18 has f = -1 at x = [0.50000001]
2021-05-26 18:37:45,224 - root - INFO - Function eval 19 at point 19 has f = -1 at x = [0.5]
2021-05-26 18:37:45,225 - root - INFO - Did a total of 1 run(s)
2021-05-26 18:37:45,232 - pyswarms.single.global_best - INFO - Optimize for 180 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}

...

2021-05-26 19:05:40,105 - root - INFO - Function eval 11 at point 11 has f = -0.999993492870232 at x = [0.49849827]
2021-05-26 19:05:40,106 - root - INFO - New rho = 0.0001 after 11 function evaluations
2021-05-26 19:05:40,109 - root - INFO - Function eval 12 at point 12 has f = -0.999999933571711 at x = [0.49984827]
2021-05-26 19:05:40,112 - root - INFO - Function eval 13 at point 13 has f = -0.999999936568442 at x = [0.50014827]
2021-05-26 19:05:40,114 - root - INFO - New rho = 1e-05 after 13 function evaluations
2021-05-26 19:05:40,117 - root - INFO - Function eval 14 at point 14 has f = -0.999999999491978 at x = [0.50001327]
2021-05-26 19:05:40,120 - root - INFO - Function eval 15 at point 15 has f = -0.999999999192305 at x = [0.49998327]
2021-05-26 19:05:40,122 - root - INFO - New rho = 1e-06 after 15 function evaluations
2021-05-26 19:05:40,124 - root - INFO - Function eval 16 at point 16 has f = -1 at x = [0.5]
2021-05-26 19:05:40,127 - root - INFO - Function eval 17 at point 17

2021-05-26 19:06:21,183 - root - INFO - New rho = 1e-06 after 12 function evaluations
2021-05-26 19:06:21,186 - root - INFO - Function eval 13 at point 13 has f = -0.999999999993511 at x = [0.4999985]
2021-05-26 19:06:21,189 - root - INFO - Function eval 14 at point 14 has f = -0.999999999996064 at x = [0.50000117]
2021-05-26 19:06:21,190 - root - INFO - New rho = 1e-07 after 14 function evaluations
2021-05-26 19:06:21,193 - root - INFO - Function eval 15 at point 15 has f = -0.999999999999935 at x = [0.49999985]
2021-05-26 19:06:21,196 - root - INFO - Function eval 16 at point 16 has f = -0.99999999999996 at x = [0.50000012]
2021-05-26 19:06:21,199 - root - INFO - New rho = 1e-08 after 16 function evaluations
2021-05-26 19:06:21,202 - root - INFO - Function eval 17 at point 17 has f = -1 at x = [0.49999999]
2021-05-26 19:06:21,205 - root - INFO - Function eval 18 at point 18 has f = -1 at x = [0.50000001]
2021-05-26 19:06:21,208 - root - INFO - Function eval 19 at point 19 has f = -1 

2021-05-26 19:06:59,480 - root - INFO - Function eval 19 at point 19 has f = -0.999999999999999 at x = [0.49999999]
2021-05-26 19:06:59,485 - root - INFO - Function eval 20 at point 20 has f = -1 at x = [0.50000001]
2021-05-26 19:06:59,487 - root - INFO - Function eval 21 at point 21 has f = -1 at x = [0.5]
2021-05-26 19:06:59,489 - root - INFO - Did a total of 1 run(s)
2021-05-26 19:06:59,494 - pyswarms.single.global_best - INFO - Optimize for 180 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████████████████████████████████████████████████████████████████████████████████████████|180/180, best_cost=-1
2021-05-26 19:07:03,388 - pyswarms.single.global_best - INFO - Optimization finished | best cost: -0.9999999999999999, best pos: [0.50000001]
2021-05-26 19:07:03,402 - GP - INFO - initializing Y
2021-05-26 19:07:03,403 - GP - INFO - initializing inference method
2021-05-26 19:07:03,404 - GP - INFO - adding kernel and likelihood as parameters
2021-05-

2021-05-26 19:07:43,803 - GP - INFO - initializing Y
2021-05-26 19:07:43,804 - GP - INFO - initializing inference method
2021-05-26 19:07:43,805 - GP - INFO - adding kernel and likelihood as parameters
2021-05-26 19:07:56,715 - root - INFO - Function eval 1 at point 1 has f = -0.913027503127989 at x = [0.67184764]
2021-05-26 19:07:56,716 - root - INFO - Initialising (coordinate directions)
2021-05-26 19:07:56,718 - root - INFO - Function eval 2 at point 2 has f = -0.774776526605444 at x = [0.77184764]
2021-05-26 19:07:56,720 - root - INFO - Function eval 3 at point 3 has f = -0.985053690825079 at x = [0.57184764]
2021-05-26 19:07:56,721 - root - INFO - Beginning main loop
2021-05-26 19:07:56,724 - root - INFO - Function eval 4 at point 4 has f = -0.999505728178218 at x = [0.51308747]
2021-05-26 19:07:56,726 - root - INFO - New rho = 0.01 after 4 function evaluations
2021-05-26 19:07:56,729 - root - INFO - Function eval 5 at point 5 has f = -0.999989445930861 at x = [0.49808747]
2021-05

([100, 100, 100, 100, 100], [True, True, True, True, True])

## Uniform entropy

In [110]:
def uni_entropy(x):
    u,l = max(x), min(x)
    return -np.log2(np.exp(st.randint(l,u).entropy()))

maximum_x = [0,200]
maximum_f = np.log2(200)

parameters_domain = [
    {"domain": (0,100)},
    {"domain": (100,200)}
]
evaluate(uni_entropy, parameters_domain, maximum_f, maximum_x)

2021-05-26 20:27:50,772 - root - INFO - Function eval 1 at point 1 has f = -6.44141112759207 at x = [ 45.82401997 132.73166139]
2021-05-26 20:27:50,772 - root - INFO - Initialising (coordinate directions)
2021-05-26 20:27:50,775 - root - INFO - Function eval 2 at point 2 has f = -6.20230948175623 at x = [ 59.09718611 132.73166139]
2021-05-26 20:27:50,778 - root - INFO - Function eval 3 at point 3 has f = -6.64646233614658 at x = [ 45.82401997 146.00482753]
2021-05-26 20:27:50,780 - root - INFO - Function eval 4 at point 4 has f = -6.64646233614658 at x = [ 32.55085383 132.73166139]
2021-05-26 20:27:50,782 - root - INFO - Function eval 5 at point 5 has f = -6.20230948175623 at x = [ 45.82401997 119.45849525]
2021-05-26 20:27:50,783 - root - INFO - Beginning main loop
2021-05-26 20:27:50,786 - root - INFO - Function eval 6 at point 6 has f = -6.89345358229344 at x = [ 35.69405786 154.58158723]
2021-05-26 20:27:50,790 - root - INFO - Function eval 7 at point 7 has f = -7.54395698076929 at

2021-05-26 20:28:06,508 - root - INFO - Function eval 20 at point 20 has f = -7.64385495517332 at x = [1.71152023e-04 2.00000000e+02]
2021-05-26 20:28:06,510 - root - INFO - New rho = 1.14101e-05 after 20 function evaluations
2021-05-26 20:28:06,514 - root - INFO - Function eval 21 at point 21 has f = -7.64385606631463 at x = [  0.         199.99998288]
2021-05-26 20:28:06,518 - root - INFO - Function eval 22 at point 22 has f = -7.64385606631463 at x = [1.71152023e-05 2.00000000e+02]
2021-05-26 20:28:06,519 - root - INFO - New rho = 1.14101e-06 after 22 function evaluations
2021-05-26 20:28:06,522 - root - INFO - Function eval 23 at point 23 has f = -7.64385617742872 at x = [  0.         199.99999829]
2021-05-26 20:28:06,526 - root - INFO - Function eval 24 at point 24 has f = -7.64385617742872 at x = [1.71152023e-06 2.00000000e+02]
2021-05-26 20:28:06,528 - root - INFO - New rho = 1.06818e-07 after 24 function evaluations
2021-05-26 20:28:06,532 - root - INFO - Function eval 25 at po

2021-05-26 20:28:36,867 - root - INFO - Function eval 6 at point 6 has f = -6.29911811364883 at x = [ 85.81129554 164.55638821]
2021-05-26 20:28:36,871 - root - INFO - Function eval 7 at point 7 has f = -6.98772960971112 at x = [ 47.83188281 174.74783549]
2021-05-26 20:28:36,875 - root - INFO - Function eval 8 at point 8 has f = -7.153468112374 at x = [ 34.35127671 176.71799522]
2021-05-26 20:28:36,879 - root - INFO - Function eval 9 at point 9 has f = -7.24388760532054 at x = [ 26.40020837 177.97516562]
2021-05-26 20:28:36,882 - root - INFO - New rho = 1.42203 after 9 function evaluations
2021-05-26 20:28:36,886 - root - INFO - Function eval 10 at point 10 has f = -7.29598256993435 at x = [ 21.59813877 178.7464077 ]
2021-05-26 20:28:36,890 - root - INFO - Function eval 11 at point 11 has f = -7.3668737674814 at x = [  4.46588773 169.52898054]
2021-05-26 20:28:36,895 - root - INFO - Function eval 12 at point 12 has f = -7.51559120239891 at x = [  0.        182.9862205]
2021-05-26 20:28

2021-05-26 20:28:52,420 - root - INFO - Function eval 20 at point 20 has f = -7.64275734492505 at x = [1.52274244e-01 2.00000000e+02]
2021-05-26 20:28:52,423 - root - INFO - Function eval 21 at point 21 has f = -7.64330687196832 at x = [7.61371219e-02 2.00000000e+02]
2021-05-26 20:28:52,425 - root - INFO - New rho = 0.0101516 after 21 function evaluations
2021-05-26 20:28:52,429 - root - INFO - Function eval 22 at point 22 has f = -7.64374634294475 at x = [  0.         199.98477258]
2021-05-26 20:28:52,433 - root - INFO - Function eval 23 at point 23 has f = -7.64374634294475 at x = [1.52274244e-02 2.00000000e+02]
2021-05-26 20:28:52,434 - root - INFO - New rho = 0.00101516 after 23 function evaluations
2021-05-26 20:28:52,437 - root - INFO - Function eval 24 at point 24 has f = -7.64384520546809 at x = [  0.         199.99847726]
2021-05-26 20:28:52,440 - root - INFO - Function eval 25 at point 25 has f = -7.64384520546809 at x = [1.52274244e-03 2.00000000e+02]
2021-05-26 20:28:52,442

2021-05-26 20:29:08,034 - root - INFO - Did a total of 1 run(s)
2021-05-26 20:29:08,041 - pyswarms.single.global_best - INFO - Optimize for 180 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|███████████████████████████████████████████████████████████████████████████████████████████|180/180, best_cost=-7.63
2021-05-26 20:29:11,974 - pyswarms.single.global_best - INFO - Optimization finished | best cost: -7.633234830844715, best pos: [6.10406099e-02 1.98594014e+02]
2021-05-26 20:29:11,987 - GP - INFO - initializing Y
2021-05-26 20:29:11,988 - GP - INFO - initializing inference method
2021-05-26 20:29:11,988 - GP - INFO - adding kernel and likelihood as parameters
2021-05-26 20:29:23,129 - root - INFO - Function eval 1 at point 1 has f = -5.07941885220837 at x = [ 99.76920058 133.58015538]
2021-05-26 20:29:23,130 - root - INFO - Initialising (coordinate directions)
2021-05-26 20:29:23,132 - root - INFO - Function eval 2 at point 2 has f = -5.06953700057156 a

...

2021-05-26 20:52:04,561 - root - INFO - Function eval 25 at point 25 has f = -7.3117280133699 at x = [ 33.30213363 192.17489795]
2021-05-26 20:52:04,565 - root - INFO - Function eval 26 at point 26 has f = -7.44501777484545 at x = [ 25.74964624 200.        ]
2021-05-26 20:52:04,570 - root - INFO - Function eval 27 at point 27 has f = -7.4574886280566 at x = [ 24.23687346 200.        ]
2021-05-26 20:52:04,573 - root - INFO - Function eval 28 at point 28 has f = -7.48174738326839 at x = [ 15.84676887 194.59032019]
2021-05-26 20:52:04,577 - root - INFO - Function eval 29 at point 29 has f = -7.52404314221897 at x = [ 15.93861972 200.        ]
2021-05-26 20:52:04,581 - root - INFO - Function eval 30 at point 30 has f = -7.53497851570872 at x = [ 14.53816609 200.        ]
2021-05-26 20:52:04,585 - root - INFO - Function eval 31 at point 31 has f = -7.54082109376498 at x = [ 13.78556607 200.        ]
2021-05-26 20:52:04,589 - root - INFO - Function eval 32 at point 32 has f = -7.464736512948

2021-05-26 20:52:20,400 - root - INFO - Function eval 17 at point 17 has f = -7.64385480822267 at x = [1.91523691e-04 2.00000000e+02]
2021-05-26 20:52:20,403 - root - INFO - Function eval 18 at point 18 has f = -7.64385480822267 at x = [  0.         199.99980848]
2021-05-26 20:52:20,405 - root - INFO - New rho = 1.27682e-05 after 18 function evaluations
2021-05-26 20:52:20,408 - root - INFO - Function eval 19 at point 19 has f = -7.64385605161958 at x = [  0.         199.99998085]
2021-05-26 20:52:20,412 - root - INFO - Function eval 20 at point 20 has f = -7.64385605161958 at x = [1.91523691e-05 2.00000000e+02]
2021-05-26 20:52:20,413 - root - INFO - New rho = 1.27682e-06 after 20 function evaluations
2021-05-26 20:52:20,417 - root - INFO - Function eval 21 at point 21 has f = -7.64385617595921 at x = [1.91523691e-06 2.00000000e+02]
2021-05-26 20:52:20,420 - root - INFO - Function eval 22 at point 22 has f = -7.64385617595921 at x = [  0.         199.99999808]
2021-05-26 20:52:20,421 

2021-05-26 20:52:50,732 - root - INFO - Function eval 5 at point 5 has f = -6.59963032981554 at x = [ 15.51699457 112.49800164]
2021-05-26 20:52:50,733 - root - INFO - Beginning main loop
2021-05-26 20:52:50,737 - root - INFO - Function eval 6 at point 6 has f = -7.12541822488403 at x = [  6.15917171 145.78463184]
2021-05-26 20:52:50,741 - root - INFO - Function eval 7 at point 7 has f = -7.61030831051675 at x = [  0.         195.40293278]
2021-05-26 20:52:50,743 - root - INFO - New rho = 1.24998 after 7 function evaluations
2021-05-26 20:52:50,746 - root - INFO - Function eval 8 at point 8 has f = -7.64385618977472 at x = [  0. 200.]
2021-05-26 20:52:50,751 - root - INFO - Function eval 9 at point 9 has f = -7.63026736755278 at x = [  1.87496669 200.        ]
2021-05-26 20:52:50,754 - root - INFO - Function eval 10 at point 10 has f = -7.63026736755278 at x = [  0.         198.12503331]
2021-05-26 20:52:50,757 - root - INFO - Function eval 11 at point 11 has f = -7.63707777783557 at x

2021-05-26 20:53:06,009 - root - INFO - Function eval 23 at point 23 has f = -7.64385618290322 at x = [9.52593501e-07 2.00000000e+02]
2021-05-26 20:53:06,011 - root - INFO - New rho = 1.127e-07 after 23 function evaluations
2021-05-26 20:53:06,014 - root - INFO - Function eval 24 at point 24 has f = -7.64385618855529 at x = [  0.         199.99999983]
2021-05-26 20:53:06,017 - root - INFO - Function eval 25 at point 25 has f = -7.64385618916501 at x = [  0.         199.99999992]
2021-05-26 20:53:06,018 - root - INFO - New rho = 1e-08 after 25 function evaluations
2021-05-26 20:53:06,021 - root - INFO - Function eval 26 at point 26 has f = -7.64385618966652 at x = [1.5e-08 2.0e+02]
2021-05-26 20:53:06,024 - root - INFO - Function eval 27 at point 27 has f = -7.64385618972062 at x = [7.5e-09 2.0e+02]
2021-05-26 20:53:06,027 - root - INFO - Function eval 28 at point 28 has f = -7.64385618977472 at x = [  0. 200.]
2021-05-26 20:53:06,028 - root - INFO - Did a total of 1 run(s)
2021-05-26 2

([0, 0, 0, 0, 0], [False, False, False, False, False])

## Normal entropy

In [15]:
def normal_MI(x):
    mu1, sigma1, mu2, sigma2 = np.abs(x)
    a = st.norm(mu1, sigma1).rvs(10_000)
    b = st.norm(mu2,sigma2).rvs(10_000)
    o = (a+b)/2
    return -mutual_info_regression(a.reshape(-1,1),o,discrete_features=False)[0]

maximum_x = [10,15,10,0.1]
maximum_f = normal_MI(maximum_x)

parameters_domain = [
    {"domain": (0,100)},
    {"domain": (0.1,15)},
    {"domain": (0,100)},
    {"domain": (0.1,15)}
]
evaluate(normal_MI, parameters_domain, maximum_f, maximum_x)